In [0]:
from pyspark.sql import Row 
from pyspark.sql.types import *
import pandas as pdd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import os
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
Train = spark.read.csv("/FileStore/tables/train.csv",header=True, inferSchema=True)
Train.show(2)

+-------------------+--------------+--------------------+---------+----------+--------------+------------------+-----------------+----------------+
 Dates| Category| Descript|DayOfWeek|PdDistrict| Resolution| Address| X| Y|
+-------------------+--------------+--------------------+---------+----------+--------------+------------------+-----------------+----------------+
2015-05-13 23:53:00| WARRANTS| WARRANT ARREST|Wednesday| NORTHERN|ARREST, BOOKED|OAK ST / LAGUNA ST|-122.425891675136|37.7745985956747|
2015-05-13 23:53:00|OTHER OFFENSES|TRAFFIC VIOLATION...|Wednesday| NORTHERN|ARREST, BOOKED|OAK ST / LAGUNA ST|-122.425891675136|37.7745985956747|
+-------------------+--------------+--------------------+---------+----------+--------------+------------------+-----------------+----------------+
only showing top 2 rows

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
win = Window().orderBy(Train.Dates)
df_imb = Train.withColumn("Id", row_number().over(win))

In [0]:
Train_c=df_imb.select("Id","Category")
df_Train_c=Train_c.groupBy("Id").pivot("Category").count().fillna(0)
Train_col=df_Train_c.columns
Train_col

Out[5]: ['Id',
 'ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LARCENY/THEFT',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'OTHER OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES FORCIBLE',
 'SEX OFFENSES NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'VEHICLE THEFT',
 'WARRANTS',
 'WEAPON LAWS']

In [0]:
df_imb.show()

+-------------------+--------------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+---+
 Dates| Category| Descript|DayOfWeek|PdDistrict| Resolution| Address| X| Y| Id|
+-------------------+--------------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+---+
2003-01-06 00:01:00| LARCENY/THEFT|GRAND THEFT FROM ...| Monday| INGLESIDE| NONE|600 Block of EDNA ST| -122.447363507104| 37.7319475636101| 1|
2003-01-06 00:01:00| LARCENY/THEFT|GRAND THEFT FROM ...| Monday| SOUTHERN| NONE| 5TH ST / FOLSOM ST| -122.403390364804| 37.780265577696| 2|
2003-01-06 00:01:00| VANDALISM|MALICIOUS MISCHIE...| Monday| SOUTHERN| NONE|TOWNSEND ST / 2ND ST|-122.39053140418699| 37.78060707982429| 3|
2003-01-06 00:01:00|FORGERY/COUNTERFE...|CHECKS, FORGERY (...| Monday| BAYVIEW| NONE|1800 Block of NEW...| -122.394925721424|37.738211541051996| 4|
2003-01-06 00:15:00| ROBBERY|ROBBERY ON THE ST...| Monday| TARAVAL| NONE|FARALLONES ST / C...| -122.459033097784| 37.7140562686562| 5|
2003-01-06 00:20:00| ASSAULT|ATTEMPTED HOMICID...| Monday| BAYVIEW|ARREST, BOOKED|1500 Block of SHA...| -122.389768984951| 37.7305636925712| 6|
2003-01-06 00:20:00| OTHER OFFENSES| PAROLE VIOLATION| Monday| BAYVIEW|ARREST, BOOKED|1500 Block of SHA...| -122.389768984951| 37.7305636925712| 7|
2003-01-06 00:31:00| OTHER OFFENSES|DRIVERS LICENSE, ...| Monday| RICHMOND| ARREST, CITED|CLEMENT ST / 14TH AV| -122.472984835661| 37.7825523645525| 8|
2003-01-06 00:33:00| ASSAULT|INFLICT INJURY ON...| Monday| MISSION| NONE|2800 Block of FOL...| -122.41407332511| 37.7516845989529| 9|
2003-01-06 00:40:00| OTHER OFFENSES|DRIVERS LICENSE, ...| Monday| NORTHERN| ARREST, CITED|POLK ST / CALIFOR...|-122.42069168079901|37.790577071030796| 10|
2003-01-06 00:42:00| WARRANTS|ENROUTE TO OUTSID...| Monday|TENDERLOIN|ARREST, BOOKED|TAYLOR ST / GEARY ST| -122.411518820366| 37.78694089985761| 11|
2003-01-06 00:55:00| ASSAULT| BATTERY| Monday| NORTHERN| NONE|1300 Block of WEB...| -122.431046366089| 37.7830295716044| 12|
2003-01-06 00:55:00| LARCENY/THEFT|PETTY THEFT SHOPL...| Monday| NORTHERN| NONE|1300 Block of WEB...| -122.431046366089| 37.7830295716044| 13|
2003-01-06 00:55:00| VANDALISM|MALICIOUS MISCHIE...| Monday| NORTHERN| NONE|1300 Block of WEB...| -122.431046366089| 37.7830295716044| 14|
2003-01-06 01:30:00| VANDALISM|MALICIOUS MISCHIE...| Monday| RICHMOND| ARREST, CITED|1000 Block of 22N...|-122.39166839300199| 37.7577932955113| 15|
2003-01-06 01:30:00| TRESPASS| TRESPASSING| Monday| RICHMOND| ARREST, CITED|1000 Block of 22N...|-122.39166839300199| 37.7577932955113| 16|
2003-01-06 01:36:00| OTHER OFFENSES|DRIVERS LICENSE, ...| Monday| NORTHERN| ARREST, CITED|GEARY BL / FRANKL...| -122.423031175095| 37.785481874719| 17|
2003-01-06 01:50:00| ASSAULT| BATTERY| Monday| BAYVIEW| NONE| 3RD ST / NEWCOMB AV|-122.39041695551701| 37.7355926106176| 18|
2003-01-06 01:54:00| OTHER OFFENSES| PROBATION VIOLATION| Monday|TENDERLOIN|ARREST, BOOKED|1400 Block of GOL...|-122.43442328150299| 37.7791934909101| 19|
2003-01-06 01:54:00|SEX OFFENSES FORC...|FORCIBLE RAPE, BO...| Monday|TENDERLOIN|ARREST, BOOKED|1400 Block of GOL...|-122.43442328150299| 37.7791934909101| 20|
+-------------------+--------------------+--------------------+---------+----------+--------------+--------------------+-------------------+------------------+---+
only showing top 20 rows

In [0]:
df = df_imb.select( "Id","Category", "PdDistrict", "Resolution","Address","DayOfWeek",minute(df_imb.Dates).alias("Minute").cast("double"),unix_timestamp(df_imb.Dates).alias('timestam').cast("double"),hour(df_imb.Dates).alias("Hour").cast("double"),month(df_imb.Dates).alias("Month").cast("double"),df_imb.X.cast("double"), df_imb.Y.cast("double"))
df.printSchema()

root
-- Id: integer (nullable = true)
-- Category: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Resolution: string (nullable = true)
-- Address: string (nullable = true)
-- DayOfWeek: string (nullable = true)
-- Minute: double (nullable = true)
-- timestam: double (nullable = true)
-- Hour: double (nullable = true)
-- Month: double (nullable = true)
-- X: double (nullable = true)
-- Y: double (nullable = true)

In [0]:
#df=df.filter(col('PdDistrict') == "SOUTHERN")
#df.count()

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.window import Window
df_cat=df.groupBy("Category").agg(count("*").alias("Numbers_Crime")).orderBy("Numbers_Crime",ascending=False)
df_percent_cat  = df_cat.withColumn('Crime_Percent', f.col('Numbers_Crime')/f.sum('Numbers_Crime').over(Window.partitionBy()))
df_percent_cat=df_percent_cat.orderBy('Crime_Percent', ascending=False)
df_percent_cat.show()

+--------------------+-------------+--------------------+
 Category|Numbers_Crime| Crime_Percent|
+--------------------+-------------+--------------------+
 LARCENY/THEFT| 174900| 0.19919161686876244|
 OTHER OFFENSES| 126182| 0.1437072418509673|
 NON-CRIMINAL| 92304| 0.10512397371900657|
 ASSAULT| 76876| 0.08755320033392214|
 DRUG/NARCOTIC| 53971| 0.06146695685548301|
 VEHICLE THEFT| 53781| 0.06125056802069133|
 VANDALISM| 44725| 0.05093679282135735|
 WARRANTS| 42214| 0.04807704353629467|
 BURGLARY| 36755| 0.04185985064614845|
 SUSPICIOUS OCC| 31414|0.035777046611293904|
 MISSING PERSON| 25989| 0.02959857593368935|
 ROBBERY| 23000| 0.02619443789583497|
 FRAUD| 16679| 0.01899552302889702|
FORGERY/COUNTERFE...| 10609|0.012082469201604922|
 SECONDARY CODES| 9985|0.011371802712604878|
 WEAPON LAWS| 8555|0.009743192008646442|
 PROSTITUTION| 7484|0.008523442313583866|
 TRESPASS| 7326|0.008343497914125522|
 STOLEN PROPERTY| 4540| 0.00517055426291699|
SEX OFFENSES FORC...| 4388|0.004997443195083645|
+--------------------+-------------+--------------------+
only showing top 20 rows

In [0]:
#df=df.join(df_percent_cat, ['Category'], how='full')
#df.show()

In [0]:
#df = df.orderBy('Crime_Percent', ascending=False).drop("Numbers_Crime") 
#df.show()

In [0]:
#df_SOUTHERN_per_cat=df_SOUTHERN_percent_cat.select(collect_list('Crime_Percent')).collect()[0][0]

In [0]:
from pyspark.sql.functions import *
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, VectorSlicer, OneHotEncoderEstimator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator , MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit , CrossValidator

In [0]:
encoding_var = [i[0] for i in df.dtypes if (i[1]=='string') & (i[0]!='Category') & (i[0]!='Resolution') ]
print("encoding_var:",encoding_var)

num_var = [i[0] for i in df.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='Category') & (i[0]!='idx') & (i[0]!='Crime_Percent')]
print("num_var:",num_var)

encoding_var: ['PdDistrict', 'Address', 'DayOfWeek']
num_var: ['Id', 'Minute', 'timestam', 'Hour', 'Month', 'X', 'Y']

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder , OneHotEncoderEstimator 
from pyspark.ml.feature import IndexToString
from matplotlib import pyplot as plt
from pyspark.ml.feature import MinMaxScaler

assemblers_scaled = [VectorAssembler(inputCols=[col], outputCol=col + "_vecc") for col in num_var]
scalers = [MinMaxScaler(inputCol=col + "_vecc", outputCol=col + "_scaled") for col in num_var]

string_indexes = [StringIndexer(inputCol = c, outputCol = 'IDX_' + c, handleInvalid = 'keep') for c in encoding_var]

onehot_indexes = [OneHotEncoderEstimator(inputCols = ['IDX_' + c], outputCols = ['OHE_' + c]) for c in encoding_var]

label_indexes = StringIndexer(inputCol = 'Category', outputCol = 'label',handleInvalid = 'keep')
labels = label_indexes.fit(df).labels

label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labels)

assembler = VectorAssembler(inputCols = num_var + ['OHE_' + c for c in encoding_var], outputCol = "features")

In [0]:
train_data, test_data = df.randomSplit([.8,.2],seed=1234)

In [0]:
#rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,numTrees=100, cacheNodeIds = True ,featureSubsetStrategy ='sqrt')
#pipe_rf = Pipeline(stages = string_indexes + onehot_indexes + [assembler, label_indexes,rf,label_converter])

In [0]:
# set threshold for the probability
#rf.setWeightCol("Crime_Percent")

In [0]:
#model_rf = pipe_rf. fit(train_data)

In [0]:
#predictions_rf = model_rf.transform(test_data)
#evaluator_rf = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
#evaluator_rf.evaluate(predictions_rf)

In [0]:
#predictions_rf.show(5)

In [0]:
#params_rf_SOUTHERN = ParamGridBuilder().addGrid(rf_SOUTHERN.impurity, ["gini" , "entropy"]).addGrid(rf_SOUTHERN.maxDepth, [4,5,6]).addGrid(rf_SOUTHERN.numTrees, [10,20]).build()
#crossval_rf_SOUTHERN = CrossValidator(estimator=pipe_rf_SOUTHERN,estimatorParamMaps=params_rf_SOUTHERN,evaluator=MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy'),numFolds=3)


In [0]:
#mod_rf_SOUTHERN= crossval_rf_SOUTHERN.fit(train_data_SOUTHERN)

In [0]:
#print(mod_rf_SOUTHERN.bestModel.stages[-2].getImpurity())
#print(mod_rf_SOUTHERN.bestModel.stages[-2].getMaxDepth())
#print(mod_rf_SOUTHERN.bestModel.stages[-2]._java_obj.getNumTrees())

In [0]:
#train_data2 = model_rf.transform(train_data)


In [0]:
#import pandas as pd
#def ExtractFeatureImp(featureImp, dataset, featuresCol):
#    list_extract = []
    #for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
     #   list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    #    #print (dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i])
   # varlist = pd.DataFrame(list_extract)
  #  varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
 #  
#    return(varlist.sort_values('score', ascending = False))

In [0]:
#varlist = ExtractFeatureImp(model_rf.stages[-2].featureImportances, train_data2, "features")
#varlist

In [0]:
#varidx = [x for x in varlist['idx'][0:10]]
#varidx

In [0]:
#slicer = VectorSlicer(inputCol="features", outputCol="features2", indices=varidx)
#train_data3 = slicer.transform(train_data2)

In [0]:
#train_data3 = train_data3.drop('rawPrediction', 'probability', 'prediction')
#model_rf2 = RandomForestClassifier(labelCol="label", featuresCol="features2", seed = 8464,numTrees=100, cacheNodeIds = True,featureSubsetStrategy ='sqrt')
#mod_rf2 = model_rf2.fit(train_data3)
#df4 = mod_rf2.transform(train_data3)

In [0]:
#predictions_rf = model_rf.transform(test_data)
#test_data2 = predictions_rf.drop('rawPrediction', 'probability', 'prediction')
#test_data3 = slicer.transform(test_data2)
#predictions_rf3 = mod_rf2.transform(test_data3)
#evaluator_rf = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
#evaluator_rf.evaluate(predictions_rf3)

In [0]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label',elasticNetParam = 1)
pipe_lr = Pipeline(stages = assemblers_scaled + scalers + string_indexes + onehot_indexes + [assembler, label_indexes,lr,label_converter])

In [0]:
#lr.setWeightCol("Crime_Percent")

In [0]:
mod_lr= pipe_lr.fit(train_data)

In [0]:
predictions_lr = mod_lr.transform(test_data)
evaluator_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
evaluator_lr.evaluate(predictions_lr)

Out[87]: 0.2764902551462869

In [0]:
Test = spark.read.csv("/FileStore/tables/test.csv",header=True, inferSchema=True)
Test.show(5)

+---+-------------------+---------+----------+--------------------+-------------------+----------------+
 Id| Dates|DayOfWeek|PdDistrict| Address| X| Y|
+---+-------------------+---------+----------+--------------------+-------------------+----------------+
 0|2015-05-10 23:59:00| Sunday| BAYVIEW|2000 Block of THO...|-122.39958770418998|37.7350510103906|
 1|2015-05-10 23:51:00| Sunday| BAYVIEW| 3RD ST / REVERE AV| -122.391522893042|37.7324323864471|
 2|2015-05-10 23:50:00| Sunday| NORTHERN|2000 Block of GOU...| -122.426001954961|37.7922124386284|
 3|2015-05-10 23:45:00| Sunday| INGLESIDE|4700 Block of MIS...| -122.437393972517|37.7214120621391|
 4|2015-05-10 23:45:00| Sunday| INGLESIDE|4700 Block of MIS...| -122.437393972517|37.7214120621391|
+---+-------------------+---------+----------+--------------------+-------------------+----------------+
only showing top 5 rows

In [0]:
#Test_SOUTHERN=Test.filter(col('PdDistrict') == "SOUTHERN")
#Test_SOUTHERN.count()

In [0]:
Test.printSchema()

root
-- Id: integer (nullable = true)
-- Dates: timestamp (nullable = true)
-- DayOfWeek: string (nullable = true)
-- PdDistrict: string (nullable = true)
-- Address: string (nullable = true)
-- X: double (nullable = true)
-- Y: double (nullable = true)

In [0]:
Test = Test.select("Id",minute(Test.Dates).alias("Minute"),unix_timestamp(Test.Dates).alias('timestam').cast("double"),hour(Test.Dates).alias("Hour"),month(Test.Dates).alias("Month").cast("double"),"Address","DayOfWeek","PdDistrict",Test.X.cast("double"), Test.Y.cast("double"))
Test.show(5)

+---+------+------------+----+-----+--------------------+---------+----------+-------------------+----------------+
 Id|Minute| timestam|Hour|Month| Address|DayOfWeek|PdDistrict| X| Y|
+---+------+------------+----+-----+--------------------+---------+----------+-------------------+----------------+
 0| 59|1.43130234E9| 23| 5.0|2000 Block of THO...| Sunday| BAYVIEW|-122.39958770418998|37.7350510103906|
 1| 51|1.43130186E9| 23| 5.0| 3RD ST / REVERE AV| Sunday| BAYVIEW| -122.391522893042|37.7324323864471|
 2| 50| 1.4313018E9| 23| 5.0|2000 Block of GOU...| Sunday| NORTHERN| -122.426001954961|37.7922124386284|
 3| 45| 1.4313015E9| 23| 5.0|4700 Block of MIS...| Sunday| INGLESIDE| -122.437393972517|37.7214120621391|
 4| 45| 1.4313015E9| 23| 5.0|4700 Block of MIS...| Sunday| INGLESIDE| -122.437393972517|37.7214120621391|
+---+------+------------+----+-----+--------------------+---------+----------+-------------------+----------------+
only showing top 5 rows

In [0]:
p4=mod_lr.transform(Test)


In [0]:
#p=model_rf.transform(Test)
#p2 = p.drop('rawPrediction', 'probability', 'prediction')
#p3 = slicer.transform(p2)
#p4 = mod_rf2.transform(p3)

In [0]:
p4.show(1)

+---+------+------------+----+-----+--------------------+---------+----------+-------------------+----------------+-------+-----------+--------------+---------+----------+--------------------+------------------+--------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------+-----------+-------------+--------------+--------------------+-------------+--------------------+--------------------+--------------------+----------+--------------+
 Id|Minute| timestam|Hour|Month| Address|DayOfWeek|PdDistrict| X| Y|Id_vecc|Minute_vecc| timestam_vecc|Hour_vecc|Month_vecc| X_vecc| Y_vecc| Id_scaled|Minute_scaled| timestam_scaled|Hour_scaled| Month_scaled| X_scaled| Y_scaled|IDX_PdDistrict|IDX_Address|IDX_DayOfWeek|OHE_PdDistrict| OHE_Address|OHE_DayOfWeek| features| rawPrediction| probability|prediction|predictedLabel|
+---+------+------------+----+-----+--------------------+---------+----------+-------------------+----------------+-------+-----------+--------------+---------+----------+--------------------+------------------+--------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------+-----------+-------------+--------------+--------------------+-------------+--------------------+--------------------+--------------------+----------+--------------+
 0| 59|1.43130234E9| 23| 5.0|2000 Block of THO...| Sunday| BAYVIEW|-122.39958770418998|37.7350510103906| [0.0]| [59.0]|[1.43130234E9]| [23.0]| [5.0]|[-122.39958770418...|[37.7350510103906]|[-4.5555751698375...| [1.0]|[0.9993358818393087]| [1.0]|[0.36363636363636...|[0.05664083112021...|[5.18836749407943...| 3.0| 2508.0| 6.0|(10,[3],[1.0])|(22682,[2508],[1.0])|(7,[6],[1.0])|(22706,[1,2,3,4,5...|[553.791489809986...|[0.08970416604696...| 1.0|OTHER OFFENSES|
+---+------+------------+----+-----+--------------------+---------+----------+-------------------+----------------+-------+-----------+--------------+---------+----------+--------------------+------------------+--------------------+-------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------+-----------+-------------+--------------+--------------------+-------------+--------------------+--------------------+--------------------+----------+--------------+
only showing top 1 row

In [0]:
p5=p4.select("Id","predictedLabel")
p5.show(2)

+---+--------------+
 Id|predictedLabel|
+---+--------------+
 0|OTHER OFFENSES|
 1|OTHER OFFENSES|
+---+--------------+
only showing top 2 rows

In [0]:
p6=p5.groupBy("Id").pivot("predictedLabel").count().fillna(0)
p6.show(5)

+------+-------+--------+------------------+---------------------------+-------------+-----------+----------------------+-----+-------------+-----------+--------------+------------+--------------+------------+-----------------+-------+-------+---------------+---------------+-------+--------------+--------+---------+-------------+--------+-----------+
 Id|ASSAULT|BURGLARY|DISORDERLY CONDUCT|DRIVING UNDER THE INFLUENCE|DRUG/NARCOTIC|DRUNKENNESS|FORGERY/COUNTERFEITING|FRAUD|LARCENY/THEFT|LIQUOR LAWS|MISSING PERSON|NON-CRIMINAL|OTHER OFFENSES|PROSTITUTION|RECOVERED VEHICLE|ROBBERY|RUNAWAY|SECONDARY CODES|STOLEN PROPERTY|SUICIDE|SUSPICIOUS OCC|TRESPASS|VANDALISM|VEHICLE THEFT|WARRANTS|WEAPON LAWS|
+------+-------+--------+------------------+---------------------------+-------------+-----------+----------------------+-----+-------------+-----------+--------------+------------+--------------+------------+-----------------+-------+-------+---------------+---------------+-------+--------------+--------+---------+-------------+--------+-----------+
 78120| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
117500| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
206378| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
212297| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
222048| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+------+-------+--------+------------------+---------------------------+-------------+-----------+----------------------+-----+-------------+-----------+--------------+------------+--------------+------------+-----------------+-------+-------+---------------+---------------+-------+--------------+--------+---------+-------------+--------+-----------+
only showing top 5 rows

In [0]:
len(p6.columns)

Out[97]: 27

In [0]:
#Kaggle Submission

In [0]:

Train_col

Out[99]: ['Id',
 'ARSON',
 'ASSAULT',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY',
 'DISORDERLY CONDUCT',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'FORGERY/COUNTERFEITING',
 'FRAUD',
 'GAMBLING',
 'KIDNAPPING',
 'LARCENY/THEFT',
 'LIQUOR LAWS',
 'LOITERING',
 'MISSING PERSON',
 'NON-CRIMINAL',
 'OTHER OFFENSES',
 'PORNOGRAPHY/OBSCENE MAT',
 'PROSTITUTION',
 'RECOVERED VEHICLE',
 'ROBBERY',
 'RUNAWAY',
 'SECONDARY CODES',
 'SEX OFFENSES FORCIBLE',
 'SEX OFFENSES NON FORCIBLE',
 'STOLEN PROPERTY',
 'SUICIDE',
 'SUSPICIOUS OCC',
 'TREA',
 'TRESPASS',
 'VANDALISM',
 'VEHICLE THEFT',
 'WARRANTS',
 'WEAPON LAWS']

In [0]:
len(Train_col)

Out[100]: 40

In [0]:
unique_list=[]
for k in Train_col:
  if k not in p6.columns:
    unique_list.append(k)
unique_list

Out[101]: ['ARSON',
 'BAD CHECKS',
 'BRIBERY',
 'EMBEZZLEMENT',
 'EXTORTION',
 'FAMILY OFFENSES',
 'GAMBLING',
 'KIDNAPPING',
 'LOITERING',
 'PORNOGRAPHY/OBSCENE MAT',
 'SEX OFFENSES FORCIBLE',
 'SEX OFFENSES NON FORCIBLE',
 'TREA']

In [0]:
#unique_list=[]
#for k in Train_col:
#  if (k !='Id'):
    
#    try:
#      if k not in p6.columns:
        
#        unique_list.append(k)
#    except:
#      
#      print('Error returned')
    

In [0]:
len(unique_list)

Out[103]: 13

In [0]:
for i in unique_list:
  p6 = p6.withColumn(i,lit(0))
#p6.show(2)

In [0]:
len(p6.columns)

Out[105]: 40

In [0]:
p6=p6.select(*[Train_col])

In [0]:
final_dataset=p6.orderBy("Id")
final_dataset.show(2)


+---+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
 Id|ARSON|ASSAULT|BAD CHECKS|BRIBERY|BURGLARY|DISORDERLY CONDUCT|DRIVING UNDER THE INFLUENCE|DRUG/NARCOTIC|DRUNKENNESS|EMBEZZLEMENT|EXTORTION|FAMILY OFFENSES|FORGERY/COUNTERFEITING|FRAUD|GAMBLING|KIDNAPPING|LARCENY/THEFT|LIQUOR LAWS|LOITERING|MISSING PERSON|NON-CRIMINAL|OTHER OFFENSES|PORNOGRAPHY/OBSCENE MAT|PROSTITUTION|RECOVERED VEHICLE|ROBBERY|RUNAWAY|SECONDARY CODES|SEX OFFENSES FORCIBLE|SEX OFFENSES NON FORCIBLE|STOLEN PROPERTY|SUICIDE|SUSPICIOUS OCC|TREA|TRESPASS|VANDALISM|VEHICLE THEFT|WARRANTS|WEAPON LAWS|
+---+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
only showing top 2 rows

In [0]:
print((final_dataset.count(), len(final_dataset.columns)))

(884262, 40)

In [0]:
final_dataset.where(col("Id")==882000).show()

+------+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
 Id|ARSON|ASSAULT|BAD CHECKS|BRIBERY|BURGLARY|DISORDERLY CONDUCT|DRIVING UNDER THE INFLUENCE|DRUG/NARCOTIC|DRUNKENNESS|EMBEZZLEMENT|EXTORTION|FAMILY OFFENSES|FORGERY/COUNTERFEITING|FRAUD|GAMBLING|KIDNAPPING|LARCENY/THEFT|LIQUOR LAWS|LOITERING|MISSING PERSON|NON-CRIMINAL|OTHER OFFENSES|PORNOGRAPHY/OBSCENE MAT|PROSTITUTION|RECOVERED VEHICLE|ROBBERY|RUNAWAY|SECONDARY CODES|SEX OFFENSES FORCIBLE|SEX OFFENSES NON FORCIBLE|STOLEN PROPERTY|SUICIDE|SUSPICIOUS OCC|TREA|TRESPASS|VANDALISM|VEHICLE THEFT|WARRANTS|WEAPON LAWS|
+------+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+
882000| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+------+-----+-------+----------+-------+--------+------------------+---------------------------+-------------+-----------+------------+---------+---------------+----------------------+-----+--------+----------+-------------+-----------+---------+--------------+------------+--------------+-----------------------+------------+-----------------+-------+-------+---------------+---------------------+-------------------------+---------------+-------+--------------+----+--------+---------+-------------+--------+-----------+

In [0]:
dbutils.fs.rm("/FileStore/df",True)

Out[110]: False

In [0]:
final_dataset.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("dbfs:/FileStore/df/final_dataset.csv")